In [93]:
#! /usr/bin/python3.5

In [94]:
import os
#dir_path = os.path.dirname(os.path.realpath(__file__))
#path_data_source=dir_path+"/data/database/csv/"
path_data_source="/home/fsg/Desktop/csv/"

sub_path_data_source_tfidf="sub_tfidf/"
sub_path_data_source_sim="semantics/sim/"

file_path_tf_idf=path_data_source+sub_path_data_source_tfidf

file_path_sim=path_data_source+sub_path_data_source_sim



file_names_tf_idf = [os.path.join(file_path_tf_idf, f) 
                      for f in os.listdir(file_path_tf_idf) 
                      if f.endswith(".csv")]

file_names_sim = [os.path.join(file_path_sim, f) 
                      for f in os.listdir(file_path_sim) 
                      if f.endswith(".csv")]


In [95]:
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic as wnic
from nltk.tokenize import word_tokenize

def similarity_by_infocontent(sense1, sense2, option):
    #sense1="Synset('"+sense1+"')"
    #sense2="Synset('"+sense2+"')"
    #print(sense1,sense2)
    sense1 = wn.synset(sense1)
    sense2 = wn.synset(sense2)
    #print(sense1,sense2)
    """ Returns similarity scores by information content. """
    #if sense1.pos != sense2.pos: # infocontent sim can't do diff POS.
        #return 0

    info_contents = ['ic-bnc-add1.dat', 'ic-bnc-resnik-add1.dat', 
                     'ic-bnc-resnik.dat', 'ic-bnc.dat', 

                     'ic-brown-add1.dat', 'ic-brown-resnik-add1.dat', 
                     'ic-brown-resnik.dat', 'ic-brown.dat', 

                     'ic-semcor-add1.dat', 'ic-semcor.dat',

                     'ic-semcorraw-add1.dat', 'ic-semcorraw-resnik-add1.dat', 
                     'ic-semcorraw-resnik.dat', 'ic-semcorraw.dat', 

                     'ic-shaks-add1.dat', 'ic-shaks-resnik.dat', 
                     'ic-shaks-resnink-add1.dat', 'ic-shaks.dat', 

                     'ic-treebank-add1.dat', 'ic-treebank-resnik-add1.dat', 
                     'ic-treebank-resnik.dat', 'ic-treebank.dat']

    if option in ['res', 'resnik']:
        #return wn.res_similarity(sense1, sense2, wnic.ic('ic-bnc-resnik-add1.dat'))
        #print('simRe snik (c1,c2) = -log p(lso(c1,c2)) = IC(lso(c1,c2)')
        return wn.res_similarity(sense1, sense2, wnic.ic('ic-treebank-resnik-add1.dat'))
    #return min(wn.res_similarity(sense1, sense2, wnic.ic(ic)) \
    #             for ic in info_contents)

    elif option in ['jcn', "jiang-conrath"]:
        #return wn.jcn_similarity(sense1, sense2, wnic.ic('ic-bnc-add1.dat'))
        #print('sim(jcn) (c1,c2 )= (IC(c1) + IC(c2 )) - 2IC(lso(c1,c2 ))')
        return wn.jcn_similarity(sense1, sense2, wnic.ic('ic-treebank.dat'))

    elif option in ['lin']:
        #return wn.lin_similarity(sense1, sense2, wnic.ic('ic-bnc-add1.dat'))
        #print('sim(lin) (c1,c2)=(2IC(lso(c1,c2 )))/(IC(c1)+IC(c2))')
        return wn.lin_similarity(sense1, sense2, wnic.ic('ic-treebank.dat'))

def sim(sense1, sense2, option="path"):
    """ Calculates similarity based on user's choice. """
    option = option.lower()
    if option.lower() in ["path", "path_similarity", 
                        "wup", "wupa", "wu-palmer", "wu-palmer",
                        'lch', "leacock-chordorow"]:
        return similarity_by_path(sense1, sense2, option) 
    elif option.lower() in ["res", "resnik",
                          "jcn","jiang-conrath",
                          "lin"]:
        return similarity_by_infocontent(sense1, sense2, option)

def max_similarity(context_sentence, ambiguous_word, option="path", 
                   pos=None, best=True):
    """
    Perform WSD by maximizing the sum of maximum similarity between possible 
    synsets of all words in the context sentence and the possible synsets of the 
    ambiguous words (see http://goo.gl/XMq2BI):
    {argmax}_{synset(a)}(\sum_{i}^{n}{{max}_{synset(i)}(sim(i,a))}
    """
    result = {}
    for i in wn.synsets(ambiguous_word):
        try:
            if pos and pos != str(i.pos()):
                continue
        except:
            if pos and pos != str(i.pos):
                continue
        result[i] = sum(max([sim(i,k,option) for k in wn.synsets(j)]+[0]) \
                        for j in word_tokenize(context_sentence))

    if option in ["res","resnik"]: # lower score = more similar
        result = sorted([(v,k) for k,v in result.items()])
    else: # higher score = more similar
        result = sorted([(v,k) for k,v in result.items()],reverse=True)
    #print (result)
    if best: return result[0][1];
    return result

In [131]:
similarity_by_infocontent('acquiring.n.01', 'acquiring.n.01', 'res')

6.1697386575732045

In [97]:
def read_cvs_by_pands(path_data_source,file_databbase,index_col, header):
    import pandas as pd
    return pd.read_csv(path_data_source+file_databbase,index_col=index_col,header=header)


In [98]:
def is_file_exist(file_path,file_name):
    file_names = [os.path.join(file_path, f) 
                      for f in os.listdir(file_path) 
                      if f.endswith(".csv")]
    #print(file_names)
    if file_name in file_names:
        return True
    else:
        return False
    


In [99]:
#is_file_exist(file_path_sim,file_path_sim+"fd.csv")

In [100]:
def add_row_csv(path_data_source,file_name,list_data):
    import csv
    with open(path_data_source+file_name, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(list_data)
        f.close()

In [101]:
def create_file(path_data_source,file_name):
    import csv
    outfile = open(path_data_source+file_name,'w')
    writer=csv.writer(outfile)
    outfile.close()

In [102]:
#create_file(file_path_sim,"ff.csv")

In [146]:
def sim_terms_one_file(path_data_source,sub_path_data_source_tfidf,file_name):
    list_terms=read_cvs_by_pands(path_data_source,sub_path_data_source_tfidf+file_name,0,0).index
    #compare between the same file
    #print(list_terms)
    #index=1
    limt=5
    for i in range(len(list_terms[:limt])):
        term=list_terms[i]
        is_term_new=False
        if not is_file_exist(file_path_sim,file_path_sim+term+".csv"): # if !Fale this new term
                #print(" not term",term)
                #create_file(file_path_sim,term+".csv")  
                is_term_new=True

        for term_next in list_terms[i:limt]:
            print("terrm",term,"term_next",term_next)

            is_term_next_new=False



            if not is_file_exist(file_path_sim,file_path_sim+term_next+".csv"): #next term is new
                    #print(" not termin",list_terms[index])
                    #create_file(file_path_sim,term_next+".csv")
                    is_term_next_new=True

            #print(term,is_term_new,term_next,is_term_next_new)
            sim=0
            list_term=[]
            list_term_next=[]
            if is_term_new or is_term_next_new:
                sim=similarity_by_infocontent(term, term_next, 'res')
                #print(term,term_next,"sim",sim)
                if sim <1:
                    sim=0
                if sim !=0:
                    list_term=[term,sim]
                    list_term_next=[term_next,sim]

            #print(list_term)
            #print(list_term_next)
            if sim !=0:
                if term != term_next:
                    print("term != term_next")
                    if is_term_new:  
                        print("             is_term_new",term)
                        create_file(file_path_sim,term+".csv")
                        add_row_csv(file_path_sim,term+".csv",list_term)#add the same sim
                        add_row_csv(file_path_sim,term+".csv",list_term_next)#add the next sim
                        is_term_new=False
                    else:
                        print("             is_term_old",term)
                        add_row_csv(file_path_sim,term+".csv",list_term_next)#add the next sim


                    if is_term_next_new:
                        print("             is_term_next_new",term_next)

                        sim_nex=similarity_by_infocontent(term_next, term_next, 'res')
                        create_file(file_path_sim,term_next+".csv")
                        list_next=[term_next,sim_nex]
                        add_row_csv(file_path_sim,term_next+".csv",list_next)#add term to next
                        add_row_csv(file_path_sim,term_next+".csv",list_term)#add term to next
                        is_term_next_new=False


                    else:
                        print("              is_term_next_old",term_next,sim)
                        add_row_csv(file_path_sim,term_next+".csv",list_term)#add term to next
                else:
                    print("term == term_next",term,term_next,sim,list_term)
                    if is_term_new:  
                        print("            is_term_new",term,sim,list_term)
                        create_file(file_path_sim,term+".csv")
                        add_row_csv(file_path_sim,term+".csv",list_term)#add the same sim
                        is_term_new=False


        print("finesed")

            
            
            
            
            
        
        
   
        
        

terrm ace.n.03 term_next ace.n.03
terrm ace.n.03 term_next acquiring.n.01
terrm ace.n.03 term_next building.n.04
terrm ace.n.03 term_next business.n.04
terrm ace.n.03 term_next care.n.04
finesed
terrm acquiring.n.01 term_next acquiring.n.01
terrm acquiring.n.01 term_next building.n.04
terrm acquiring.n.01 term_next business.n.04
terrm acquiring.n.01 term_next care.n.04
finesed
terrm building.n.04 term_next building.n.04
terrm building.n.04 term_next business.n.04
terrm building.n.04 term_next care.n.04
finesed
terrm business.n.04 term_next business.n.04
terrm business.n.04 term_next care.n.04
finesed
terrm care.n.04 term_next care.n.04
term == term_next care.n.04 care.n.04 10.690011115357335 ['care.n.04', 10.690011115357335]
            is_term_new care.n.04 10.690011115357335 ['care.n.04', 10.690011115357335]
finesed
